# Demo sonoUno server

In this notebook, we will use the sonoUno servers APIs to
- login the test user in order to retrieve an access token
- create a transform
- execute a job
- play the output


### Initialisation

In [ ]:
import requests
from pathlib import Path
from pprint import pprint
from sonounolib import Track

SERVER_URL = DATA_URL = 'http://api.sonouno.org.ar'
EMAIL = "test_email@test.com"
PASSWORD = 'password'

### Access token
We login a test user, which already exists in the sonoUno User database. Then the access token is stored in the request session that will be used from now on.

In [ ]:
user = {
    'email': EMAIL,
    'password': PASSWORD,
}
response = requests.post(f'{SERVER_URL}/iam/login', json=user)
response.raise_for_status()
access_token = response.json()['access_token']
session = requests.Session()
session.headers['Authorization'] = f'Bearer {access_token}'

### The source code of the transform

In [ ]:
SOURCE = Path('shortwav.py').read_text()
print(SOURCE)

### Creation of the transform

In [ ]:
transform_in = {
    'name': "Test transformation",
    'public': True,
    'language': "python",
    'source': SOURCE,
    'entry_point': {"name": "pipeline"}
}
response = session.post(f'{SERVER_URL}/transforms', json=transform_in)
response.raise_for_status()
transform = response.json()
pprint(transform)

### Creation and execution of the job

In [ ]:
job_in = {
    'transform_id': transform['_id'],
    'inputs': [
        {
            'id': 'pipeline.repeat',
            'value': 2,
        },
    ],
}
response = session.post(f'{SERVER_URL}/jobs', json=job_in)
response.raise_for_status()
job = response.json()
pprint(job['outputs'])


### Access and play the sonification


In [ ]:
uri = job['outputs'][0]['value']
print(f'The audio file is available at: {uri}')
track = Track.load(uri)
track.play()